### Ejercicio 16: Planta

Una compañía fabrica dos tipos de productos, uno normal y uno extra. La compañía tiene dos
plantas de fabricacón, una en Toledo y otra en Málaga. La planta de Toledo tiene un presupuesto
de operacón diario de 45000 euros y puede producir como mucho 300 unidades (entre los dos
tipos de productos). El coste de fabricar el producto normal en la planta de Toledo es de 150
euros mientras que el producto extra cuesta 200 euros fabricarlo. La planta de Málaga tiene
un presupuesto diario de 36000 euros y puede fabricar como mucho 250 unidades (entre los dos
productos). Los costes de fabricacón en esta planta son de 135 euros para el producto normal
y de 180 euros para el producto extra. La compañía quiere limitar la produccón diaria a 250
unidades de producto normal y 350 unidades de producto extra. El beneficio de venta de cada
unidad de proudcto normal es de 50 euros mientras que el del producto extra es de 70 euros por
unidad. Se pide calcular el máximo beneficio y qué cantidad de cada tipo de unidad (normal y
extra) se debe producir en cada planta de fabricacón.

In [9]:
from pyomo.environ import *
import pandas as pd
model = ConcreteModel()

In [15]:
fab = ['Toledo', 'Málaga']
prod = ['Normal', 'Extra']
model.f = Set(initialize = fab)
model.p = Set(initialize = prod)

Cost = [{'Toledo':150, 'Málaga':135}, {'Toledo':200, 'Málaga':180}]
Costes = pd.DataFrame(Cost, index=prod)
Costes
Benef = [50, 70.1]
Beneficio = pd.DataFrame(Benef, index=prod)

maxpr = [250, 350]
maxprod = pd.DataFrame(maxpr, index=prod)

maxpl = [300, 250]
maxplantas = pd.DataFrame(maxpl, index=fab)

maxpres = [45000, 36000]
maxpresupuesto = pd.DataFrame(maxpres, index=fab)

Costes=Costes.to_dict()
Beneficio=Beneficio.to_dict()
maxprod=maxprod.to_dict()
maxplantas=maxplantas.to_dict()
maxpresupuesto=maxpresupuesto.to_dict()

Costes


    'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
    This is usually indicative of a modelling error. To avoid this warning,
    use block.del_component() and block.add_component().
    'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
    This is usually indicative of a modelling error. To avoid this warning,
    use block.del_component() and block.add_component().


,Toledo,Málaga
Normal,150,135
Extra,200,180


In [11]:
model.F = Var(model.p, model.f, domain=NonNegativeReals)

In [12]:
model.profit = Objective(expr = sum(sum(model.F[p,f] for f in model.f)*Beneficio[0][p] for p in model.p),
                         sense=maximize)

In [13]:
model.maxproduccion = ConstraintList()
for p in model.p:
    model.maxproduccion.add(
        sum(model.F[p,f] for f in model.f) <= maxprod[0][p]
    )

model.maxplantas = ConstraintList()
for f in model.f:
    model.maxplantas.add (
        sum(model.F[p,f] for p in model.p) <= maxplantas[0][f]
    )

model.maxpresp = ConstraintList()
for f in model.f:
    model.maxpresp.add(
        sum(model.F[p,f]*Costes[f][p] for p in model.p) <= maxpresupuesto[0][f]
    )


In [14]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

6 Set Declarations
    F_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    p*f :    4 : {('Normal', 'Toledo'), ('Normal', 'Málaga'), ('Extra', 'Toledo'), ('Extra', 'Málaga')}
    f : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Toledo', 'Málaga'}
    maxplantas_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    maxpresp_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    maxproduccion_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {1, 2}
    p : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    2 : {'Normal', 'E